In [1]:
!pip install xmltodict

In [2]:
# May need to update data rate limit for downloads:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import xml.etree.ElementTree as ET
import xmltodict
from collections import MutableMapping 

In [ ]:
# Using Salesforce (CRM) as test case
crm = '1108524'
search = requests.get('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-'.format(crm))
searchSoup = BeautifulSoup(search.content)

In [ ]:
# Identify document table
searchTable = pd.read_html(str(searchSoup.findAll('table')))[2].iloc[:8,:]
searchTable['accNo'] = searchTable.apply(lambda x: re.findall('\d+-\d+-\d+',x['Description'])[0], axis=1)
searchTable.head(1)

,Filings,Format,Description,Filing Date,File/Film Number,accNo
0,10-Q,Documents Interactive Data,Quarterly report [Sections 13 or 15(d)]Acc-no:...,2020-08-28,001-32224201144893,0001108524-20-000041


In [ ]:
# Helper functions to parse data and documents
def convert_flatten(d, parent_key ='', sep ='_'): 
    items = [] 
    for k, v in d.items(): 
        new_key = parent_key + sep + k if parent_key else k 
  
        if isinstance(v, MutableMapping): 
            items.extend(convert_flatten(v, new_key, sep = sep).items()) 
        else: 
            items.append((new_key, v)) 
    return dict(items)

def getFSdoc(CIK,acc_no):
    documents = requests.get('https://www.sec.gov/Archives/edgar/data/{}/{}/{}'\
                             .format(CIK,acc_no.replace('-',''), acc_no + '-index.htm'))
    docSoup = BeautifulSoup(documents.content)
    docTables = pd.read_html(str(docSoup.findAll('table')))
    financials = requests.get('https://www.sec.gov/Archives/edgar/data/{}/{}/{}'\
                              .format(CIK,acc_no.replace('-',''),docTables[0].loc[0,'Document'].replace(' iXBRL','')))
    fsSoup = BeautifulSoup(financials.content)
    instFile = docTables[1][docTables[1]['Description'].str.contains('XBRL INSTANCE DOCUMENT')].iloc[0,2]
    print('https://www.sec.gov/Archives/edgar/data/{}/{}/{}'\
                            .format(CIK,acc_no.replace('-',''),instFile))
    instance = requests.get('https://www.sec.gov/Archives/edgar/data/{}/{}/{}'\
                            .format(CIK,acc_no.replace('-',''),instFile))
    instSoup = BeautifulSoup(re.sub('xbrli:','',instance.text))
    
    # Parse 'context' entries and store in df
    xbrl = xmltodict.parse(str(instSoup))['html']['body']['xbrl']
    ctxt_df = pd.DataFrame([convert_flatten(d) for d in xbrl['context']])
    
    # Parse 'fact' entries, add context and create df
    [t.attrs.update({t.name.split(':')[0]:t.name.split(':')[1],'text':t.text}) for t\
         in instSoup.findAll(re.compile(r'us-gaap|crm')) if 'contextref' in t.attrs.keys()]
    dtl_df = pd.DataFrame([t.attrs for t in instSoup.findAll(re.compile(r'us-gaap|crm'))\
                           if 'contextref' in t.attrs.keys()])
    xbrl_df = ctxt_df.merge(dtl_df,left_on='@id',right_on='contextref')
    return xbrl_df

In [ ]:
# Iterate SEC documents
xbrl_dict = {}
for idx, row in searchTable.iterrows():
    doc = row['Filings']+'_'+row['Filing Date']
    xbrl_dict[doc] = getFSdoc(crm,row['accNo'])
    xbrl_dict[doc+'_glbl'] = xbrl_dict[doc][xbrl_dict[doc]\
        ['entity_segment_xbrldi:explicitmember_@dimension'] == 'srt:StatementGeographicalAxis']\
            [list(xbrl_dict[doc].columns[:8])+['text']]

https://www.sec.gov/Archives/edgar/data/1108524/000110852420000041/crm-20200731_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852420000029/crm-20200430_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852420000014/crm-20200131_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852419000076/crm-20191031_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852419000052/crmq2fy2010-q_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852419000025/crmq1fy2010-q_htm.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852419000009/crm-20190131.xml
https://www.sec.gov/Archives/edgar/data/1108524/000110852418000087/crm-20181031.xml


In [ ]:
glbl_df = None
for key in xbrl_dict.keys():
    if 'glbl' in key:
        if glbl_df is None:
            glbl_df = xbrl_dict[key]
        else:
            glbl_df = pd.concat([glbl_df,xbrl_dict[key]])

In [ ]:
glbl_df = glbl_df[[c for c in glbl_df.columns if 'date' in c or 'text' in c]]
glbl_df.columns = ['entity','start','end','segment','amount']
glbl_df = glbl_df.drop_duplicates().reset_index(drop=True)

In [ ]:
glbl_df['amount'] = glbl_df['amount'].astype(float)
glbl_df['segment'] = glbl_df['segment'].str.replace('srt:','')
glbl_df['segment'] = glbl_df['segment'].str.replace('Member','')
glbl_df.to_csv('hist_rev_by_seg.csv')

In [ ]:
glbl_df

,entity,start,end,segment,amount
0,0001108524,2020-05-01,2020-07-31,Americas,3.596000e+09
1,0001108524,2019-05-01,2019-07-31,Americas,2.816000e+09
2,0001108524,2020-02-01,2020-07-31,Americas,6.966000e+09
3,0001108524,2019-02-01,2019-07-31,Americas,5.433000e+09
4,0001108524,2020-05-01,2020-07-31,Europe,1.070000e+09
5,0001108524,2019-05-01,2019-07-31,Europe,7.860000e+08
6,0001108524,2020-02-01,2020-07-31,Europe,2.104000e+09
7,0001108524,2019-02-01,2019-07-31,Europe,1.541000e+09
8,0001108524,2020-05-01,2020-07-31,AsiaPacific,4.850000e+08
9,0001108524,2019-05-01,2019-07-31,AsiaPacific,3.950000e+08


## API Example
Using SEC-API.io, requires a subscription for more than a small number of requests.

In [ ]:
# Sign up for API here
# package used to execute HTTP POST request to the API
import json
import urllib.request
import pprint

# API Key
with open('sec_api_cred.json') as f:
    cred = json.loads(f.read()) # replace YOUR_API_KEY with the API key you got from sec-api.io after sign up
# API endpoint
API = "https://api.sec-api.io?token=" + cred['token']

# define the filter parameters you want to send to the API 
payload = {
  "query": { "query_string": { "query": "ticker:crm AND filedAt:{2016-01-01 TO 2016-12-31} AND formType:\"10-Q\"" } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

# format your payload to JSON bytes
jsondata = json.dumps(payload)
jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes

# Only 100 free API calls - don't waste them
"""
# instantiate the request 
req = urllib.request.Request(API)

# set the correct HTTP header: Content-Type = application/json
req.add_header('Content-Type', 'application/json; charset=utf-8')
# set the correct length of your request
req.add_header('Content-Length', len(jsondataasbytes))

# send the request to the API
response = urllib.request.urlopen(req, jsondataasbytes)

# read the response 
res_body = response.read()
# transform the response into JSON
filings = json.loads(res_body.decode("utf-8"))

# print JSON 
print(filings)
"""

FileNotFoundError: ignored